In [1]:
#pip install biopython
#pip install beautifulsoup4
import pandas as pd
from pandas import DataFrame
from Bio import Entrez
import time
from Bio.Entrez import efetch
from bs4 import BeautifulSoup
import csv   
import os

In [2]:
################  10/14/21 by ED  ##########################################################

In [3]:
def pmid_fetch(PMID):
    Entrez.email = "ezhou@bentley.edu"  
    handle = efetch(db='pubmed', id=PMID, retmode='xml')
    xml_data = handle.read()
    soup = BeautifulSoup(xml_data)

    ##### Extract from XML to holder lists 
    abstract_records = []
    year_records=[]
    #######################pubmedbookarticle#########################
    for tag in soup.findAll("pubmedbookarticle"):

        abstract_tags = tag.findAll('abstract')
        if len(abstract_tags) == 0:
            abstract_records.append({"pmid": int(tag.pmid.text), "abstracttext": 'no_data'})
        else:
            for abstract_tag in abstract_tags:
                b_rec = {}
                try:
                    b_rec['pmid'] = int(tag.pmid.text)
                    b_rec['abstracttext'] = abstract_tag.text 
                except (AttributeError, RuntimeError, TypeError, NameError):
                    b_rec['abstracttext']='missing_or_blank_line'
                abstract_records.append(b_rec)
                
    for tag in soup.findAll("pubmedbookarticle"):    
        year_tags = tag.findAll('accessionnumber')
        if len(year_tags) != 0:
            for year_tag in year_tags:
                y_rec = {}
                try:
                    y_rec['pmid'] = int(tag.pmid.text)
                    y_rec['pub_year'] = year_tag.year.text  
                except (AttributeError, RuntimeError, TypeError, NameError):
                    try:
                        y_rec['pmid'] = int(tag.pmid.text)
                        y_rec['pub_year'] = year_tag.medlinedate.text
                    except (AttributeError, RuntimeError, TypeError, NameError):
                        y_rec['pub_year']='missing_or_blank_line'                    
                year_records.append(y_rec)        
        else:    
            for year_tag in tag.findAll('pubmedpubdate'):
                y_rec = {}
                try:
                    y_rec['pmid'] = int(tag.pmid.text)
                    y_rec['pub_year'] = year_tag.year.text  
                except (AttributeError, RuntimeError, TypeError, NameError):
                    y_rec['pub_year']='missing_or_blank_line'
                year_records.append(y_rec)

    ###########################pubmedarticle######################
    for tag in soup.findAll("pubmedarticle"):

        abstract_tags = tag.findAll('abstract')
        if len(abstract_tags) == 0:
            abstract_records.append({"pmid": int(tag.pmid.text), "abstracttext": 'no_data'})
        else:
            for abstract_tag in abstract_tags:
                b_rec = {}
                try:
                    b_rec['pmid'] = int(tag.pmid.text)
                    b_rec['abstracttext'] = abstract_tag.text 
                except (AttributeError, RuntimeError, TypeError, NameError):
                    b_rec['abstracttext']='missing_or_blank_line'
                abstract_records.append(b_rec)
                
    for tag in soup.findAll("pubmedarticle"):    
        year_tags = tag.findAll('accessionnumber')
        if len(year_tags) != 0:
            for year_tag in year_tags:
                y_rec = {}
                try:
                    y_rec['pmid'] = int(tag.pmid.text)
                    y_rec['pub_year'] = year_tag.text  
                except (AttributeError, RuntimeError, TypeError, NameError):
                    try:
                        y_rec['pmid'] = int(tag.pmid.text)
                        y_rec['pub_year'] = year_tag.medlinedate.text
                    except (AttributeError, RuntimeError, TypeError, NameError):
                        y_rec['pub_year']='missing_or_blank_line'                    
                year_records.append(y_rec)        
        else:    
            for year_tag in tag.findAll('pubmedpubdate'):
                y_rec = {}
                try:
                    y_rec['pmid'] = int(tag.pmid.text)
                    y_rec['pub_year'] = year_tag.text  
                except (AttributeError, RuntimeError, TypeError, NameError):
                    y_rec['pub_year']='missing_or_blank_line'
                year_records.append(y_rec)

    ####### post processing
    ##abstract_frame
    abstract_frame = DataFrame(abstract_records,columns=['pmid','abstracttext'])
    abstract_frame=abstract_frame.drop_duplicates()
    ##NCT number
    pubyear_frame = DataFrame(year_records,columns=['pmid','pub_year'])
    #pubyear_frame_first = pubyear_frame[pubyear_frame.groupby('pmid')['pub_year'].transform('min') == pubyear_frame['pub_year']]
    pubyear_frame = pubyear_frame.drop_duplicates()
    
    ###### data consolidation

    abstract_frame = abstract_frame.replace('\n',' ', regex=True)
    abstract_frame=abstract_frame.drop_duplicates()
    
    TOTAL_frame=pd.merge(abstract_frame,pubyear_frame,how='outer')
    
    TOTAL_frame.to_csv('NCT_Raw_DrugFull.csv',mode='a',index=None,header=False)

In [4]:
target_list_expand=pd.read_csv("P_B_Drug_NCTPREP.csv")
################### input target list #############################
for PMID in target_list_expand.PMID:
    pmid_fetch(PMID)

HTTPError: HTTP Error 400: Bad Request

In [ ]:
headerList = ['PMID','abstract','accessionnumber']
w0=pd.read_csv('NCT_Raw.csv',header=None)
w0=w0.drop_duplicates()
w0.to_csv('NCT_Raw.csv', header=headerList, index=False)